In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as scs
import statistics
import copy
import warnings
warnings.filterwarnings('ignore')

In [2]:
scores_df = pd.read_csv('C:/Users/Gabriel/Documents/GitHub/MIE368_PROJECT/team_proj/data/season_data.csv')
playoff_scores_df = pd.read_csv('C:/Users/Gabriel/Documents/GitHub/MIE368_PROJECT/team_proj/data/playoff_data.csv')
real_elo = pd.read_csv('C:/Users/Gabriel/Documents/GitHub/MIE368_PROJECT/team_proj/data/nhl_elo.csv')

In [3]:
scores_df = pd.read_csv(r'/content/season_data.csv')
playoff_df = pd.read_csv(r'/content/playoff_data.csv')
real_elo = pd.read_csv(r'/content/nhl_elo.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/season_data.csv'

In [3]:
scores_df.head()

,Date,Visitor,G_Vis,Home,G_Home,OT Ind,Season
0,2000-10-04,Colorado Avalanche,2.0,Dallas Stars,2.0,1,2001
1,2000-10-05,Ottawa Senators,4.0,Boston Bruins,4.0,1,2001
2,2000-10-05,Chicago Blackhawks,2.0,Buffalo Sabres,4.0,0,2001
3,2000-10-05,Detroit Red Wings,4.0,Calgary Flames,3.0,0,2001
4,2000-10-05,Vancouver Canucks,3.0,Philadelphia Flyers,6.0,0,2001


## Game Result Elo Adjustment

In [3]:
first_yr = scores_df[scores_df['Season'] == 2001]
Teams = first_yr['Home'].unique().tolist()

team_changes = {
    'Atlanta Thrashers': 'Winnipeg Jets',
    'Phoenix Coyotes': 'Arizona Coyotes',
    'Mighty Ducks of Anaheim': 'Anaheim Ducks'
    # Add more mappings as needed
}

K = 6

In [4]:
def calculate_prob_winning(home_team, away_team, elo_ratings):

    Elo_diff_home = elo_ratings[home_team]-elo_ratings[away_team] + 26  #add extra for home-ice advantage
    #Elo_diff_away = elo_ratings[away_team] - elo_ratings[home_team]
    prob_win_home = 1/(10**(-1*Elo_diff_home/400)+1)
    #prob_win_away = 1/(10**(-1*Elo_diff_away/400)+1)
    prob_win_away = 1 - prob_win_home

    return prob_win_home, prob_win_away, Elo_diff_home

In [5]:
def margin_of_victory(home_goals, away_goals):

    mov = abs(home_goals - away_goals)
    mov_mult = 0.6686*np.log(mov)+0.8048

    return mov_mult

In [6]:
# Function to update ELO ratings after a game -- Could add input for elo_ratings dictionary
def update_game_elo(home_team, away_team, home_goals, away_goals, season, df, idx, elo_ratings):

    # Map old team names to new ones
    home_team = team_changes.get(home_team, home_team)
    away_team = team_changes.get(away_team, away_team)

    #print('Home team is ' + str(home_team))
    #print('Away team is ' + str(away_team))

    if home_team not in elo_ratings:
        # New teams added past a certain year start with a different ELO rating
        elo_ratings[home_team] = 1500
    if away_team not in elo_ratings:
        elo_ratings[away_team] = 1500

     # Calculate the probability of winning the game for each team
    home_prob, away_prob, Elo_diff_home = calculate_prob_winning(home_team, away_team, elo_ratings)

    # Update DataFrame values
    df.at[idx, 'EloDiffHome'] = Elo_diff_home
    df.at[idx, 'EloDiffAway'] = -1 * Elo_diff_home
    df.at[idx, 'WinProbHome'] = home_prob
    df.at[idx, 'WinProbVis'] = away_prob

     # Update Pregame Favorite Multiplier
    if home_goals > away_goals:
        home_win = 1
        #away_win = 0
        winner_elo_diff = Elo_diff_home
    elif home_goals < away_goals:
        home_win = 0
        #away_win = 1
        winner_elo_diff = -1*Elo_diff_home
    else:
        return df, elo_ratings

    pre_g_fav_h = home_win - home_prob
    #pre_g_fav_a = away_win - away_prob

    auto_corr = 2.05/(winner_elo_diff*0.001 + 2.05)

     # Adjust ELO shift for margin of victory
    mov_multiplier = auto_corr * margin_of_victory(home_goals, away_goals)
    #print('home goals:' + str(home_goals))
    #print('away goals:' + str(away_goals))
    #print('mult is: ' + str(mov_multiplier))

     # Calculate ELO shift based on game result
    elo_shift_h = K * mov_multiplier * pre_g_fav_h
    #elo_shift_a = K * mov_multiplier * pre_g_fav_a
    #print('elo shift is: ' + str(elo_shift_h))

    elo_ratings[home_team] += elo_shift_h
    elo_ratings[away_team] -= elo_shift_h

    return df, elo_ratings

## End of Season Elo Adjustments

In [7]:
def elo_adjuster(df):

    # Initialize variables to store the final Elo ratings for the season
    final_elo_ratings = {}
    elo_ratings = {}
    season_weight = 0.7
    average_weight = 0.3
    current_season = None

    # Iterate over seasons and games
    for idx, row in df.iterrows():
        if current_season is None:
            current_season = row['Season']

        # Check if the season in the current row is different from the current season
        if row['Season'] != current_season:
            # Save the previous season's Elo ratings and calculate the starting Elo for the new season
            final_elo_ratings[current_season] = dict(elo_ratings)
            elo_ratings = {}  # Reset Elo ratings for the new season

            for team in final_elo_ratings[current_season]:
                prev_season_elo = final_elo_ratings[current_season][team]
                league_avg_elo = sum(final_elo_ratings[current_season].values()) / len(final_elo_ratings[current_season])
                starting_elo = (season_weight * prev_season_elo) + (average_weight * league_avg_elo)
                elo_ratings[team] = round(starting_elo, 2)

            current_season = row['Season']

        # Update Elo ratings for the game
        df, elo_ratings = update_game_elo(row['Home'], row['Visitor'], row['G_Home'], row['G_Vis'], row['Season'], df, idx, elo_ratings)

    # Calculate the final Elo ratings for the last season
    final_elo_ratings[current_season] = dict(elo_ratings)

    return df, elo_ratings, final_elo_ratings

In [8]:
seasons, elo, final_season_elos = elo_adjuster(scores_df)

In [9]:
elo

{'Dallas Stars': 1550.7819539073464,
 'Colorado Avalanche': 1579.9182021884367,
 'Boston Bruins': 1642.0502212110703,
 'Ottawa Senators': 1479.0506615440481,
 'Buffalo Sabres': 1486.7120784302224,
 'Chicago Blackhawks': 1393.8606927123315,
 'Calgary Flames': 1515.3348272021358,
 'Detroit Red Wings': 1443.4317712658462,
 'Philadelphia Flyers': 1421.6856205830084,
 'Vancouver Canucks': 1485.5682678752944,
 'Arizona Coyotes': 1405.4197600448936,
 'St. Louis Blues': 1483.661524120255,
 'Edmonton Oilers': 1580.5090602196963,
 'Florida Panthers': 1538.0042767487648,
 'Anaheim Ducks': 1370.185824502506,
 'Minnesota Wild': 1543.83706875618,
 'New Jersey Devils': 1533.2411289939719,
 'Montreal Canadiens': 1410.3105469348452,
 'Pittsburgh Penguins': 1504.8509804888336,
 'Nashville Predators': 1505.5125954824891,
 'San Jose Sharks': 1386.7944419061305,
 'Tampa Bay Lightning': 1539.0902552500847,
 'New York Islanders': 1513.2474188071392,
 'Washington Capitals': 1475.2423478319913,
 'Los Angeles K

In [44]:
def forecast_game_elo(home_team, away_team, season, df, idx, elo_ratings, simulation_wins):

    # Map old team names to new ones
    home_team = team_changes.get(home_team, home_team)
    away_team = team_changes.get(away_team, away_team)

    if home_team not in elo_ratings:
        elo_ratings[home_team] = 1500
    if away_team not in elo_ratings:
        elo_ratings[away_team] = 1500

     # Calculate the probability of winning the game for each team
    home_prob, away_prob, Elo_diff_home = calculate_prob_winning(home_team, away_team, elo_ratings)

    # Update DataFrame values
    df.at[idx, 'EloDiffHome'] = Elo_diff_home
    df.at[idx, 'EloDiffAway'] = -1 * Elo_diff_home
    df.at[idx, 'WinProbHome'] = home_prob
    df.at[idx, 'WinProbVis'] = away_prob

    home_win = np.random.choice([0, 1], p=[away_prob, home_prob])

    # Calculate base scores for the home and away team
    score_home = 2.845905 + 0.003584*Elo_diff_home
    score_away = 2.845905 + 0.003584*(-1)*Elo_diff_home

    conditions_satisfied = 0

    while conditions_satisfied == 0:

        # poisson distribution to shift scores
        poisson_score_home = np.random.poisson(max(0, score_home))
        poisson_score_away = np.random.poisson(max(0, score_away))

        # Ensure non-negative scores by replacing any negative values with zero
        # poisson_score_home = np.maximum(0, poisson_score_home)
        # poisson_score_away = np.maximum(0, poisson_score_away)

        home_goal_diff = poisson_score_home - poisson_score_away

        # does it satsify conditions?
        if home_win == 1:
            if home_goal_diff > 0:
                conditions_satisfied = 1

        if home_win == 0:
            if home_goal_diff < 0:
                conditions_satisfied = 1

    #------------------------------------------------------------------------#
    OT_choice = 0

    if abs(home_goal_diff) == 1:
        # Decide if the game goes to OT
        OT_choice = np.random.choice(a= [1, 0], p= [0.505, 0.495])

    if home_win == 1:
        winner_elo_diff = Elo_diff_home
        simulation_wins[home_team] += 2

        if OT_choice == 1:
            simulation_wins[away_team] += 1

    if home_win == 0:
        winner_elo_diff = -1*Elo_diff_home
        simulation_wins[away_team] += 2

        if OT_choice == 1:
            simulation_wins[home_team] += 1

    pre_g_fav_h = home_win - home_prob
    auto_corr = 2.05/(winner_elo_diff*0.001 + 2.05)

    # Adjust ELO shift for margin of victory
    mov_multiplier = auto_corr * margin_of_victory(poisson_score_home, poisson_score_away)

    # Calculate ELO shift based on game result
    elo_shift_h = K * mov_multiplier * pre_g_fav_h

    elo_ratings[home_team] += elo_shift_h
    elo_ratings[away_team] -= elo_shift_h

    df.at[idx, 'G_Home'] = poisson_score_home
    df.at[idx, 'G_Vis'] = poisson_score_away
    df.at[idx, 'OT Ind'] = OT_choice

    #--------------------------------------------------------------------------#

    return df, elo_ratings, simulation_wins

In [45]:
# Iterate over seasons and games
def season_forecaster(df, starting_elo):

    # Create a dictionary to track point totals
    simulation_pts = {team: 0 for team in df['Home'].unique()}

    for idx, row in df.iterrows():
        df, starting_elo, simulation_pts = forecast_game_elo(row['Home'], row['Visitor'], row['Season'], df, idx, starting_elo, simulation_pts)

    return df, starting_elo, simulation_pts

In [46]:
def playoff_tracker(simulation_pts, playoffs_made):

    conference_mapping = {
        'East': ['Florida Panthers', 'Toronto Maple Leafs', 'Tampa Bay Lightning', 'Boston Bruins', 'Buffalo Sabres', 'Detroit Red Wings',
                 'Ottawa Senators', 'Montreal Canadiens', 'Carolina Hurricanes', 'New York Rangers', 'Pittsburgh Penguins',
                 'Washington Capitals', 'New York Islanders', 'Columbus Blue Jackets', 'New Jersey Devils', 'Philadelphia Flyers'],
        'West': ['Colorado Avalanche', 'Minnesota Wild', 'St. Louis Blues', 'Dallas Stars', 'Nashville Predators', 'Winnipeg Jets',
                 'Chicago Blackhawks', 'Arizona Coyotes', 'Calgary Flames', 'Edmonton Oilers', 'Los Angeles Kings', 'Vegas Golden Knights',
                 'Vancouver Canucks', 'San Jose Sharks', 'Anaheim Ducks', 'Seattle Kraken']
    }

    # Separate teams into East and West lists
    for conference, teams in conference_mapping.items():
        valid_teams = [i for i in teams if i in simulation_pts]
        sorted_teams = sorted(valid_teams, key=lambda x: simulation_pts[x], reverse=True)

        for team in sorted_teams[:8]:
            playoffs_made[team] += 1

    return playoffs_made

In [47]:
np.random.seed(0)
df2 = scores_df[scores_df['Season'] == 2022]
season_2022 = copy.deepcopy(scores_df[scores_df['Season'] == 2022])

num_simulations = 500
season_weight = 0.7
average_weight = 0.3

# Create a dictionary to track overall standings
standings = {team: np.zeros(num_simulations) for team in df2['Home'].unique()}

# Create a dictionary to track playoff appearances
playoffs_counter = {team: 0 for team in df2['Home'].unique()}

for simulation in range(num_simulations):
    #season_2022 = copy.deepcopy(scores_df[scores_df['Season'] == 2022])
    elo_end = copy.deepcopy(final_season_elos[2021])  # Creating a deep copy of final_season_elos[2021]

    elo_ratings = {}  # Reset Elo ratings for the new season

    for team in elo_end:
        prev_season_elo = elo_end[team]
        league_avg_elo = sum(elo_end.values()) / len(elo_end)
        starting_elo = (season_weight * prev_season_elo) + (average_weight * league_avg_elo)
        elo_ratings[team] = round(starting_elo, 2)

    season_2022, elo_ratings, simulation_pts = season_forecaster(season_2022, elo_ratings)
    playoffs_counter = playoff_tracker(simulation_pts, playoffs_counter)

    for team, pts in simulation_pts.items():
        standings[team][simulation] = pts


for team, pts in standings.items():
    #print(pts)
    mean_pts = np.mean(pts)
    std_deviation = np.std(pts)
    print(f"{team}: Mean point total = {mean_pts}, Standard Deviation = {std_deviation}")

Tampa Bay Lightning: Mean point total = 98.116, Standard Deviation = 12.444378007759166
Vegas Golden Knights: Mean point total = 102.626, Standard Deviation = 12.485596661753895
Anaheim Ducks: Mean point total = 77.462, Standard Deviation = 11.714800723870637
Colorado Avalanche: Mean point total = 102.234, Standard Deviation = 11.72583660128351
Edmonton Oilers: Mean point total = 94.728, Standard Deviation = 12.435675132456621
Toronto Maple Leafs: Mean point total = 96.678, Standard Deviation = 12.531173767847926
Washington Capitals: Mean point total = 97.604, Standard Deviation = 12.325550048577954
Buffalo Sabres: Mean point total = 76.22, Standard Deviation = 11.810656205308831
Carolina Hurricanes: Mean point total = 100.162, Standard Deviation = 12.041418354994564
Columbus Blue Jackets: Mean point total = 79.408, Standard Deviation = 12.312819985689712
Detroit Red Wings: Mean point total = 76.036, Standard Deviation = 11.92504524100433
Florida Panthers: Mean point total = 97.312, St

In [48]:
len(season_2022[season_2022['OT Ind'] == 1])/len(season_2022)

0.18140243902439024

In [49]:
mean_pts = pd.DataFrame(columns = ['PTS']).rename_axis('Team')

for team, pts in standings.items():
    mean_pts.loc[team] = [np.mean(pts)]



In [50]:
mean_pts.sort_values(by='PTS', ascending=False)

,PTS
Team,
Vegas Golden Knights,102.626
Colorado Avalanche,102.234
Carolina Hurricanes,100.162
Boston Bruins,100.024
Pittsburgh Penguins,98.980
Tampa Bay Lightning,98.116
Washington Capitals,97.604
Florida Panthers,97.312
Toronto Maple Leafs,96.678


In [20]:
print("Elo ratings at the end of the simulation")

elo_df = pd.DataFrame(columns = ['Elo']).rename_axis('Team')

for team, elo in elo_ratings.items():
    elo_df.loc[team] = [elo]

elo_df.sort_values(by='Elo', ascending=False)

Elo ratings at the end of the simulation


,Elo
Team,
Carolina Hurricanes,1589.771201
Winnipeg Jets,1567.384863
New York Rangers,1559.990132
Vegas Golden Knights,1556.959834
Toronto Maple Leafs,1548.572534
Washington Capitals,1547.616097
Colorado Avalanche,1543.990261
Edmonton Oilers,1538.938789
Minnesota Wild,1532.541201


In [21]:
print("Elo ratings at the end of the real prior season")

elo_df = pd.DataFrame(columns = ['Elo']).rename_axis('Team')

for team, elo in final_season_elos[2021].items():
    elo_df.loc[team] = [elo]

elo_df.sort_values(by='Elo', ascending=False)

Elo ratings at the end of the real prior season


,Elo
Team,
Vegas Golden Knights,1580.366387
Colorado Avalanche,1574.882544
Boston Bruins,1563.605372
Pittsburgh Penguins,1561.056207
Tampa Bay Lightning,1558.416560
Carolina Hurricanes,1557.582815
Washington Capitals,1553.844148
Florida Panthers,1547.747386
Toronto Maple Leafs,1543.472271


In [51]:
print("Playoff Probabilities")

playoff_df = pd.DataFrame(columns = ['Playoff Prob']).rename_axis('Team')

playoffs_counter_normalized = {team: count / num_simulations for team, count in playoffs_counter.items()}

for team, playoff_prob in playoffs_counter_normalized.items():
    playoff_df.loc[team] = [playoff_prob]

playoff_df.sort_values(by='Playoff Prob', ascending=False)

Playoff Probabilities


,Playoff Prob
Team,
Colorado Avalanche,0.858
Vegas Golden Knights,0.846
Carolina Hurricanes,0.792
Boston Bruins,0.784
Pittsburgh Penguins,0.756
Washington Capitals,0.726
Florida Panthers,0.724
Tampa Bay Lightning,0.718
Toronto Maple Leafs,0.718


In [35]:
season_2022

,Date,Visitor,G_Vis,Home,G_Home,OT Ind,Season,EloDiffHome,EloDiffAway,WinProbHome,WinProbVis
23662,2021-10-12,Pittsburgh Penguins,3.0,Tampa Bay Lightning,6.0,0,2022,24.150000,-24.150000,0.534699,0.465301
23663,2021-10-12,Seattle Kraken,4.0,Vegas Golden Knights,3.0,1,2022,82.260000,-82.260000,0.616218,0.383782
23664,2021-10-13,Winnipeg Jets,4.0,Anaheim Ducks,2.0,0,2022,-36.470000,36.470000,0.447707,0.552293
23665,2021-10-13,Chicago Blackhawks,2.0,Colorado Avalanche,5.0,0,2022,100.820000,-100.820000,0.641152,0.358848
23666,2021-10-13,Vancouver Canucks,0.0,Edmonton Oilers,2.0,0,2022,72.510000,-72.510000,0.602861,0.397139
...,...,...,...,...,...,...,...,...,...,...,...
24969,2022-04-29,San Jose Sharks,5.0,Seattle Kraken,3.0,0,2022,94.349099,-94.349099,0.632537,0.367463
24970,2022-04-29,Vegas Golden Knights,1.0,St. Louis Blues,2.0,0,2022,-6.009528,6.009528,0.491352,0.508648
24971,2022-04-29,Boston Bruins,4.0,Toronto Maple Leafs,1.0,0,2022,70.515717,-70.515717,0.600109,0.399891
24972,2022-04-29,Calgary Flames,1.0,Winnipeg Jets,2.0,1,2022,121.980540,-121.980540,0.668670,0.331330


In [36]:
scores_df[scores_df['Season'] == 2022]


,Date,Visitor,G_Vis,Home,G_Home,OT Ind,Season,EloDiffHome,EloDiffAway,WinProbHome,WinProbVis
23662,2021-10-12,Pittsburgh Penguins,6.0,Tampa Bay Lightning,2.0,0,2022,24.150000,-24.150000,0.534699,0.465301
23663,2021-10-12,Seattle Kraken,3.0,Vegas Golden Knights,4.0,0,2022,82.260000,-82.260000,0.616218,0.383782
23664,2021-10-13,Winnipeg Jets,1.0,Anaheim Ducks,4.0,0,2022,-36.470000,36.470000,0.447707,0.552293
23665,2021-10-13,Chicago Blackhawks,2.0,Colorado Avalanche,4.0,0,2022,100.820000,-100.820000,0.641152,0.358848
23666,2021-10-13,Vancouver Canucks,2.0,Edmonton Oilers,3.0,1,2022,72.510000,-72.510000,0.602861,0.397139
...,...,...,...,...,...,...,...,...,...,...,...
24969,2022-04-29,San Jose Sharks,0.0,Seattle Kraken,3.0,0,2022,28.181444,-28.181444,0.540468,0.459532
24970,2022-04-29,Vegas Golden Knights,7.0,St. Louis Blues,4.0,0,2022,67.212498,-67.212498,0.595538,0.404462
24971,2022-04-29,Boston Bruins,2.0,Toronto Maple Leafs,5.0,0,2022,30.475178,-30.475178,0.543745,0.456255
24972,2022-04-29,Calgary Flames,1.0,Winnipeg Jets,3.0,0,2022,-46.357503,46.357503,0.433679,0.566321


In [19]:
# 2022-23 Season
real_scores = np.array([['Boston Bruins',135],['Toronto Maple Leafs',111],['Tampa Bay Lightning',98],
              ['Florida Panthers',92],['Buffalo Sabres',91],['Ottawa Senators',86],['Detroit Red Wings',80],
              ['Montreal Canadiens',68],['Carolina Hurricanes',113],['New Jersey Devils',112],['New York Rangers',107],
              ['New York Islanders',93],['Pittsburgh Penguins',91],['Washington Capitals',80],['Philadelphia Flyers',75],
              ['Columbus Blue Jackets',59],['Colorado Avalanche',109],['Dallas Stars',108],['Minnesota Wild',103],
              ['Winnipeg Jets',95],['Nashville Predators',92],['St. Louis Blues',81],['Arizona Coyotes',70],
              ['Chicago Blackhawks',59],['Vegas Golden Knights',111],['Edmonton Oilers',109],['Los Angeles Kings',104],
              ['Seattle Kraken',100],['Calgary Flames',93],['Vancouver Canucks',83],['San Jose Sharks',60],['Anaheim Ducks',58]])

real_scores = pd.DataFrame(real_scores)
real_scores = real_scores.rename(columns={0:'Team', 1:'PTS'})
real_scores['PTS'] = pd.to_numeric(real_scores['PTS'])

In [52]:
# 2021-22 Season
real_scores = np.array([['Florida Panthers',122],['Toronto Maple Leafs',115],['Tampa Bay Lightning',110],
              ['Boston Bruins',107],['Buffalo Sabres',75],['Detroit Red Wings',74],['Ottawa Senators',73],
              ['Montreal Canadiens',55],['Carolina Hurricanes',116],['New York Rangers',110],['Pittsburgh Penguins',103],
              ['Washington Capitals',100],['New York Islanders',84],['Columbus Blue Jackets',81],['New Jersey Devils',63],
              ['Philadelphia Flyers',61],['Colorado Avalanche',119],['Minnesota Wild',113],['St. Louis Blues',109],
              ['Dallas Stars',98],['Nashville Predators',97],['Winnipeg Jets',89],['Chicago Blackhawks',68],
              ['Arizona Coyotes',57],['Calgary Flames',111],['Edmonton Oilers',104],['Los Angeles Kings',99],
              ['Vegas Golden Knights',94],['Vancouver Canucks',92],['San Jose Sharks',77],['Anaheim Ducks',76],['Seattle Kraken',60]])

real_scores = pd.DataFrame(real_scores)
real_scores = real_scores.rename(columns={0:'Team', 1:'PTS'})
real_scores['PTS'] = pd.to_numeric(real_scores['PTS'])

In [21]:
#2018-19 Season
real_scores = np.array([['Tampa Bay Lightning',128],['Boston Bruins',107],['Toronto Maple Leafs',100],
              ['Montreal Canadiens',96],['Florida Panthers',86],['Buffalo Sabres',76],['Detroit Red Wings',74],
              ['Ottawa Senators',64],['Washington Capitals',104],['New York Islanders',103],['Pittsburgh Penguins',100],
              ['Carolina Hurricanes',99],['Columbus Blue Jackets',98],['Philadelphia Flyers',82],['New York Rangers',78],
              ['New Jersey Devils',72],['Nashville Predators',100],['Winnipeg Jets',99],['St. Louis Blues',99],
              ['Dallas Stars',93],['Colorado Avalanche',90],['Chicago Blackhawks',84],['Minnesota Wild',83],
              ['Calgary Flames',107],['San Jose Sharks',101],['Vegas Golden Knights',93],
              ['Vancouver Canucks',81],['Anaheim Ducks',80],['Edmonton Oilers',79],['Los Angeles Kings',71]])

real_scores = pd.DataFrame(real_scores)
real_scores = real_scores.rename(columns={0:'Team', 1:'PTS'})
real_scores['PTS'] = pd.to_numeric(real_scores['PTS'])

In [53]:
real_scores.sort_values(by='PTS', ascending=False)

,Team,PTS
0,Florida Panthers,122
16,Colorado Avalanche,119
8,Carolina Hurricanes,116
1,Toronto Maple Leafs,115
17,Minnesota Wild,113
24,Calgary Flames,111
9,New York Rangers,110
2,Tampa Bay Lightning,110
18,St. Louis Blues,109
3,Boston Bruins,107


In [54]:
points_df = pd.merge(mean_pts, real_scores, on='Team')
points_df['Diff'] = (np.abs(points_df['PTS_y']-points_df['PTS_x']))
display(points_df)

,Team,PTS_x,PTS_y,Diff
0,Tampa Bay Lightning,98.116,110,11.884
1,Vegas Golden Knights,102.626,94,8.626
2,Anaheim Ducks,77.462,76,1.462
3,Colorado Avalanche,102.234,119,16.766
4,Edmonton Oilers,94.728,104,9.272
5,Toronto Maple Leafs,96.678,115,18.322
6,Washington Capitals,97.604,100,2.396
7,Buffalo Sabres,76.220,75,1.220
8,Carolina Hurricanes,100.162,116,15.838
9,Columbus Blue Jackets,79.408,81,1.592


In [55]:
mean_x = np.mean(points_df['PTS_x'])
std_dev_x = np.std(points_df['PTS_x'])

mean_y = np.mean(points_df['PTS_y'])
std_dev_y = np.std(points_df['PTS_y'])

mean_diff = np.mean(points_df['Diff'])
std_dev_diff = np.std(points_df['Diff'])

test = pd.DataFrame({'Mean':[mean_x, mean_y, mean_diff],'Std Dev':[std_dev_x, std_dev_y, std_dev_diff]})
test.rename({0:'Model', 1:'Actual', 2:'Diff'},axis=0)

,Mean,Std Dev
Model,89.652250,8.096495
Actual,91.000000,19.964030
Diff,12.556875,8.810565


In [56]:
def model_accuracy_stats(model_points_results, real_points_results):
  points_df = pd.merge(model_points_results, real_points_results, on='Team')
  points_df['Diff'] = (np.abs(points_df['PTS_y']-points_df['PTS_x']))

  data = ['ttest_ind', 'f_oneway', 'mannwhitneyu',
                     'ranksums', 'brunnermunzel', 'mood', 'ansari',
                     'ks_2samp', 'kruskal', 'fligner',
                     'levene', 'bartlett', 'median_test']
  stats = []
  p_value = []

  stats.append(scs.ttest_ind(points_df['PTS_x'], points_df['PTS_y'], equal_var=False)[0])
  stats.append(scs.f_oneway(points_df['PTS_x'], points_df['PTS_y'])[0])
  stats.append(scs.mannwhitneyu(points_df['PTS_x'], points_df['PTS_y'])[0])
  stats.append(scs.ranksums(points_df['PTS_x'], points_df['PTS_y'])[0])
  stats.append(scs.brunnermunzel(points_df['PTS_x'], points_df['PTS_y'])[0])
  stats.append(scs.mood(points_df['PTS_x'], points_df['PTS_y'])[0])
  stats.append(scs.ansari(points_df['PTS_x'], points_df['PTS_y'])[0])
  stats.append(scs.ks_2samp(points_df['PTS_x'], points_df['PTS_y'])[0])
  stats.append(scs.kruskal(points_df['PTS_x'], points_df['PTS_y'])[0])
  stats.append(scs.fligner(points_df['PTS_x'], points_df['PTS_y'])[0])
  stats.append(scs.levene(points_df['PTS_x'], points_df['PTS_y'])[0])
  stats.append(scs.bartlett(points_df['PTS_x'], points_df['PTS_y'])[0])
  stats.append(scs.median_test(points_df['PTS_x'], points_df['PTS_y'])[0])

  p_value.append(scs.ttest_ind(points_df['PTS_x'], points_df['PTS_y'], equal_var=False)[1])
  p_value.append(scs.f_oneway(points_df['PTS_x'], points_df['PTS_y'])[1])
  p_value.append(scs.mannwhitneyu(points_df['PTS_x'], points_df['PTS_y'])[1])
  p_value.append(scs.ranksums(points_df['PTS_x'], points_df['PTS_y'])[1])
  p_value.append(scs.brunnermunzel(points_df['PTS_x'], points_df['PTS_y'])[1])
  p_value.append(scs.mood(points_df['PTS_x'], points_df['PTS_y'])[1])
  p_value.append(scs.ansari(points_df['PTS_x'], points_df['PTS_y'])[1])
  p_value.append(scs.ks_2samp(points_df['PTS_x'], points_df['PTS_y'])[1])
  p_value.append(scs.kruskal(points_df['PTS_x'], points_df['PTS_y'])[1])
  p_value.append(scs.fligner(points_df['PTS_x'], points_df['PTS_y'])[1])
  p_value.append(scs.levene(points_df['PTS_x'], points_df['PTS_y'])[1])
  p_value.append(scs.bartlett(points_df['PTS_x'], points_df['PTS_y'])[1])
  p_value.append(scs.median_test(points_df['PTS_x'], points_df['PTS_y'])[1])

  data = pd.DataFrame([data, stats, p_value])
  data = data.transpose()
  data = data.rename(columns={0:'Test', 1:'Statistic', 2:'P-value'})

  MeanAE = np.sum(points_df['Diff'])/points_df['Diff'].shape[0]
  RSS = np.sum((points_df['Diff'])**2)
  print(f'The Mean Absolute Error is: {MeanAE}')
  print(f'The Residual Sum of Squares is: {RSS}')
  display(data)

In [57]:
model_accuracy_stats(mean_pts, real_scores)

The Mean Absolute Error is: 12.556875000000002
The Residual Sum of Squares is: 7529.637328000001


,Test,Statistic,P-value
0,ttest_ind,-0.348319,0.729386
1,f_oneway,0.121326,0.728781
2,mannwhitneyu,463.0,0.514902
3,ranksums,-0.657931,0.510583
4,brunnermunzel,0.591238,0.558287
5,mood,-4.931849,0.000001
6,ansari,700.0,0.000004
7,ks_2samp,0.375,0.021412
8,kruskal,0.432883,0.510578
9,fligner,18.111925,0.000021
